In [1]:
/*A*/
/*1. Adjuk meg azokat az autó márkákat melyeket legalább kétszer béreltek MárkaNév,BérlésSzám formában!*/

SELECT MarkaNev,COUNT(*) AS BerlesSzam FROM Markak m
JOIN Tipusok t ON m.MarkaID=t.MarkaID
JOIN Autok a ON t.TipusID=a.TipusID
Join Berel b ON a.AutoKod=b.AutoKod
GROUP BY MarkaNev
HAVING COUNT(*)>=2

/*2. Keressük meg azokat az auto tipusokat, melyek kevesebb mint 600 lej bevételt hoztak(Figyeljetek arra hogy esetlegesen lehet olyan auto is melynek nincs bevétele). */

SELECT TipusNev, COALESCE(SUM(NapiDij*(DATEDIFF(day,Mikortol,Meddig))), 0) AS Bevetel FROM Tipusok t
JOIN Autok a ON t.TipusID=a.TipusID
LEFT JOIN Berel b ON a.AutoKod=b.AutoKod
GROUP BY TipusNev
HAVING  COALESCE(SUM(NapiDij*(DATEDIFF(day,Mikortol,Meddig))), 0)<600

/*3. Adjuk meg azokat a bérlőket akik csak Volkswagen Polo-t béreltek.*/

SELECT DISTINCT Berlok.Nev
FROM Berlok
INNER JOIN Berel ON Berlok.BerloID = Berel.BerloID
INNER JOIN Autok ON Berel.AutoKod = Autok.AutoKod
INNER JOIN Tipusok ON Autok.TipusID = Tipusok.TipusID
INNER JOIN Markak ON Tipusok.MarkaID = Markak.MarkaID
WHERE Markak.MarkaNev = 'Volkswagen' AND Tipusok.TipusNev = 'Polo';

/*4. Adjuk meg azokat a bérlőket akik extra nélküli autót béreltek.*/

SELECT DISTINCT Berlok.Nev
FROM Berlok
JOIN Berel ON Berlok.BerloID = Berel.BerloID
JOIN Autok ON Berel.AutoKod = Autok.AutoKod
LEFT JOIN AutoExtraja ON Autok.AutoKod = AutoExtraja.AutoKod
WHERE AutoExtraja.AutoKod IS NULL;

/*5. Adjuk meg a bérlők esetén hogy melyik tipusu autót átlagban hány napra béreltek.*/

SELECT Nev,TipusNev, AVG(DATEDIFF(day,Mikortol,Meddig)) AS Atlagos_berleshossz FROM Berlok bsz
JOIN Berel b ON bsz.BerloID=b.BerloID
JOIN Autok a ON b.AutoKod=a.AutoKod
JOIN Tipusok t ON a.TipusID=t.TipusID
GROUP BY Nev,TipusNev

/*6. Adjuk meg csokkenő sorrendben minden autó tipus esetén hány kulőnböző extrával van felszerelve (MárkaNév, TipusNév, ExtrákSzáma). Figyeljetek arra, hogy azok az autók is megjelenjenek melyeknek nincs egyáltalán extra felszereltsége*/

SELECT MarkaNev, TipusNev, COUNT(ae.ExtraID) AS ExtrakSzama FROM Markak m
JOIN Tipusok t ON m.MarkaID=t.MarkaID
JOIN Autok a ON t.TipusID=a.TipusID
LEFT JOIN AutoExtraja ae ON a.AutoKod=ae.AutoKod
GROUP BY MarkaNev,TipusNev
ORDER BY ExtrakSzama DESC

/*7. Halmazmúveletekkel adjuk meg azokat az autokat (MárkaNév, TipusNev, Szin, GyártásiEv) formában 
-  Melyeket 2009 után gyártottak de nem piros szinuek .*/

SELECT MarkaNev, TipusNev, SzinNev, GyartasiEv FROM Markak m
JOIN Tipusok t ON m.MarkaID=t.MarkaID
JOIN Autok a ON t.TipusID=a.TipusID
JOIN Szinek sz ON a.SzinKod=sz.SzinKod
WHERE GyartasiEv > 2009
EXCEPT
SELECT MarkaNev, TipusNev, SzinNev, GyartasiEv FROM Markak m
JOIN Tipusok t ON m.MarkaID=t.MarkaID
JOIN Autok a ON t.TipusID=a.TipusID
JOIN Szinek sz ON a.SzinKod=sz.SzinKod
WHERE SzinNev='piros'

/*- Melyekben van Klima és van GPS is*/

SELECT MarkaNev, TipusNev, GyartasiEv FROM Markak m
JOIN Tipusok t ON m.MarkaID=t.MarkaID
JOIN Autok a ON t.TipusID=a.TipusID
JOIN AutoExtraja ae ON a.AutoKod=ae.AutoKod
JOIN Extrak e ON ae.ExtraID=e.ExtraID
WHERE ExtraNev='Klima'
INTERSECT
SELECT MarkaNev, TipusNev, GyartasiEv FROM Markak m
JOIN Tipusok t ON m.MarkaID=t.MarkaID
JOIN Autok a ON t.TipusID=a.TipusID
JOIN AutoExtraja ae ON a.AutoKod=ae.AutoKod
JOIN Extrak e ON ae.ExtraID=e.ExtraID
WHERE ExtraNev='GPS'

/*B*/
/*1. Növeljük a napi diját az összes Volkswagen márkájú autónak 10%-al.*/

UPDATE Autok
SET NapiDij = NapiDij * 1.1
FROM Autok A
JOIN Tipusok T ON A.TipusID = T.TipusID
JOIN Markak M ON T.MarkaID = M.MarkaID
WHERE M.MarkaNev = 'Volkswagen';

/*2. Töröljuk az összes 2000 ás 1990 között gyártott autót!*/

DELETE FROM Berel
FROM Berel b
JOIN Autok a ON a.AutoKod=b.AutoKod
WHERE a.GyartasiEv BETWEEN 1990 AND 2000

DELETE FROM AutoExtraja
FROM AutoExtraja ae
JOIN Autok a ON a.AutoKod=ae.AutoKod
WHERE a.GyartasiEv BETWEEN 1990 AND 2000

DELETE FROM Extrak
FROM Extrak e
JOIN AutoExtraja ae ON e.ExtraID=ae.ExtraID
JOIN Autok a ON a.AutoKod=ae.AutoKod
WHERE a.GyartasiEv BETWEEN 1990 AND 2000



DELETE FROM Autok
WHERE GyartasiEv BETWEEN 1990 AND 2000

/*3. Szúrd be a kedvenc autód az adatbázisba (legalább 2 extrával)*/

INSERT INTO Autok (Rendszam,TipusID,SzinKod,GyartasiEv,NapiDij,Csillag)
VALUES ('SJ 14 MLY',7,3,2023,250,5)

DECLARE @AutoKod INT;
SELECT @AutoKod = AutoKod FROM Autok WHERE Rendszam = 'SJ 14 MLY';

INSERT INTO AutoExtraja (AutoKod, ExtraID)
VALUES
(@AutoKod, (SELECT ExtraID FROM Extrak WHERE ExtraNev = 'Klima')),
(@AutoKod, (SELECT ExtraID FROM Extrak WHERE ExtraNev = 'GPS'));

INSERT INTO Autok (Rendszam,TipusID,SzinKod,GyartasiEv,NapiDij,Csillag)
VALUES ('SJ 88 NBT',2,1,2020,300,5)

DECLARE @AutoKod1 INT;
SELECT @AutoKod1 = AutoKod FROM Autok WHERE Rendszam = 'Sj 88 NBT';

INSERT INTO AutoExtraja (AutoKod, ExtraID)
VALUES
(@AutoKod1, (SELECT ExtraID FROM Extrak WHERE ExtraNev = 'Klima')),
(@AutoKod1, (SELECT ExtraID FROM Extrak WHERE ExtraNev = 'GPS'));

: Msg 208, Level 16, State 1, Line 4
Invalid object name 'Autok'.

Total execution time: 00:00:00.012